# Text Classification project
#### Information Retrival course
#### JCT, Data Science

Elroi Netzer
208612937
0525551587

## Flow of Work:

Steps in text classification, conducted in this notebook:
1. Loading the data
2. Pre-processing and tokenizing
3. Vocabulaty creation and vectorizing the text input
4. feed vectors into some model

In [63]:
import pandas as pd
from transformers import AutoTokenizer, BertTokenizerFast

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

from stop_words import STOP_WORDS

In [27]:
PUNKT = ["\"", "$", "%", "&", "\\", "\'", "(", ")", "*", "+", ",", "-", ".", "/", ":", ";", "<", "=", ">", "?", "@", "[", "]", "^", "_", "`", "{", "|", "}", "~", "`"]
INVALID_CHARS = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "q", "w", "e", "r", "t", "y", "u", "i", "o", "p", "a", "d", "f", "g", "h", "j", "k", "l", "z", "x", "c", "v", "b", "n", "m"]

## Loading the Data

In [4]:
data_file = r".\\chaverim_makshivim-train.xlsx"
data = pd.read_excel(data_file)

In [16]:
data.head(2)

,Seq,Site,URL,ID,Hdate,Rabby,Subject,Question,Answer,Tags,SubjectLen,QuestionLen,AnswerLen,Year,Hyear,Hmonth,Hday,Gender,1 גברים,2 נשים
0,NaN,כיפה,www.kipa.co.il/ask/show/121227-שנאת-ערבים,121227,"ד תמוז תשס""ז",חברים מקשיבים,שנאת ערבים,"\nשלום לכם, אני נער בן 14 וגר באחת ה""התנחלויות...","\nב""ה\nד´ שלום!\nראשית רציתי לומר לך שאני מאוד...",NaN,9,182,1257,2007,תשסז,תמוז,ד,1,NaN,NaN
1,NaN,כיפה,www.kipa.co.il/ask/show/296834-ברכת-המזון-ראש-...,296834,"א כסליו תשע""ד",חברים מקשיבים,ברכת המזון ראש חודש,\nלעיתים יוצא לנו להתחיל את הסעודה בראש חודש ו...,"\nב""ה\nשלום וברכה\nהדבר נתון במחלוקת הפוסקים ה...",NaN,18,286,1095,2013,תשעד,כסליו,א,1,NaN,NaN


In [17]:
data["Question"].iloc[1]

'\nלעיתים יוצא לנו להתחיל את הסעודה בראש חודש ולסיים אותה בחשכת זאת אומרת שיוצא ראש חודש השאלה האם צריך להזכיר יעלה ויבוא בברכת המזון כי נוצר מצב שהמברך כבר לא בראש חודש כי נכנס לחשכה כשכבר יצא ראש חודש.\nשאלה שניה מה קורה כאשר ראש חודש נופל בשבת וסעודה שלישית מתארכת לחשכה כשכבר יצא שבת .'

## Pre-processing and Tokenizing

In [28]:
def clean_tokens(tokens):
    def is_valid_token(tok):
        return (len(tok) > 1 and (tok not in STOP_WORDS) and ([c for c in tok if c in PUNKT + INVALID_CHARS] == []))
    return [tok for tok in tokens if is_valid_token(tok)]

In [8]:
alephbert_tokenizer = BertTokenizerFast.from_pretrained('onlplab/alephbert-base')
alephbertgimmel_tokenizer = AutoTokenizer.from_pretrained('dicta-il/alephbertgimmel-base')

In [10]:
question_tokens = [alephbert_tokenizer.tokenize(qu) for qu in data['Question']]
answer_tokens = [alephbert_tokenizer.tokenize(an) for an in data['Answer']]

In [ ]:
question_tokens[1]

In [25]:
clean_que_tok = [clean_tokens(que_toks) for que_toks in question_tokens]
clean_ans_tok = [clean_tokens(ans_toks) for ans_toks in answer_tokens]

In [ ]:
clean_que_tok[1]

## Vocabulary creation and vectorizing

In [75]:
def create_frequency_dict(texts):
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    return frequency

In [81]:
def filter_tokens_by_freq(texts, min_freq = 10):
    freq_dict = create_frequency_dict(texts)
    docs = [
        [token for token in text if freq_dict[token] >= min_freq]
        for text in texts
    ]

    return docs

In [117]:
ques_docs1 = filter_tokens_by_freq(clean_que_tok, 1)
ques_docs10 = filter_tokens_by_freq(clean_que_tok)
ques_docs100 = filter_tokens_by_freq(clean_que_tok, min_freq = 100)

vocab1 = create_frequency_dict(ques_docs1)
vocab10 = create_frequency_dict(ques_docs10)
vocab100 = create_frequency_dict(ques_docs100)

dict1 = Dictionary(ques_docs1)
dict10 = Dictionary(ques_docs10)
dict100= Dictionary(ques_docs100)

print(len(vocab1.keys()), len(dict1))
print(len(vocab10.keys()), len(dict10))
print(len(vocab100.keys()), len(dict100))

27113 27113
6780 6780
689 689


In [110]:
cv = CountVectorizer()
X = cv.fit_transform([" ".join(tokens) for tokens in ques_docs100])
X = X.toarray()

In [111]:
X.shape

(5031, 664)

In [116]:
len(set(cv.vocabulary_.keys()).difference(set(vocab100.keys())))

41

In [112]:
sorted(cv.vocabulary_.keys())

['אבא',
 'אדם',
 'אהבה',
 'אודה',
 'אוהב',
 'אוהבת',
 'אוכל',
 'אומר',
 'אומרים',
 'אומרת',
 'אח',
 'אחד',
 'אחת',
 'איסור',
 'אישית',
 'אלא',
 'אלוקים',
 'אלי',
 'אליה',
 'אליהם',
 'אליו',
 'אליי',
 'אמא',
 'אמונה',
 'אמור',
 'אמורה',
 'אמיתי',
 'אמיתית',
 'אמר',
 'אמרה',
 'אמרו',
 'אמרתי',
 'אנו',
 'אנשים',
 'אסור',
 'אפילו',
 'אפשר',
 'אצלי',
 'אשמח',
 'בא',
 'באה',
 'באול',
 'באופן',
 'באותו',
 'באמונה',
 'באמת',
 'באתר',
 'בבוקר',
 'בבית',
 'בבני',
 'בבקשה',
 'בגיל',
 'בדיוק',
 'בדרך',
 'בה',
 'בהם',
 'בהתחלה',
 'בזה',
 'בזמן',
 'בחור',
 'בחורה',
 'בחיים',
 'בטוח',
 'בטוחה',
 'בי',
 'ביום',
 'ביותר',
 'ביחד',
 'בישיבה',
 'בית',
 'בכוונה',
 'בכיתה',
 'בכך',
 'בכל',
 'בכלל',
 'בלב',
 'במה',
 'במיוחד',
 'במצב',
 'במשך',
 'בן',
 'בנאדם',
 'בנושא',
 'בנות',
 'בני',
 'בנים',
 'בס',
 'בסדר',
 'בסוף',
 'בסניף',
 'בע',
 'בעולם',
 'בעז',
 'בעיה',
 'בעיות',
 'בעיקר',
 'בעניין',
 'בעצם',
 'בעצמי',
 'בערך',
 'בצורה',
 'בקיצור',
 'בקשר',
 'בר',
 'ברא',
 'בראש',
 'ברוך',
 'ברור',
 'בשבט',
 'בשבי

In [103]:
print(len(Dictionary(clean_que_tok)))

27113


In [53]:
dict = Dictionary(clean_que_tok)
corpus = [dict.doc2bow(line) for line in clean_que_tok]

In [88]:
print(clean_que_tok[0])

['שלום', 'נער', 'בן', 'וגר', 'באחת', 'התנחלויות', 'אשמח', 'לדעת', 'בעיה', 'לשנוא', 'ערבי', 'בכמה', 'מצבים', 'שהוא', 'ערבי', 'שהוא', 'מביט', 'בי', 'במבט', '##י', 'איבה', 'כשהוא', 'מקלל', 'בתודה', 'מראש', '##´']


In [1]:
model = TfidfModel(corpus)  # fit model
vector = model[corpus[0]]

NameError: name 'TfidfModel' is not defined

In [ ]:
corpus[0]

In [ ]:
vector